In [ ]:
from ultralytics import YOLO
import os
import cv2

## Training

In [ ]:
output_dir = 'CustomOutputDir'

model = YOLO("yolo11m.yaml").load("yolo11m.pt")
model = model.to('cuda')
# customize options such as batch size, workers etc. for your environment
model.train(data = 'TrainData.yaml',
            epochs = 75, imgsz = 2560, batch = 0.9, workers = 8, project = output_dir, name = output_dir, optimizer = 'SGD', cos_lr = True, lr0 = 0.0002, lrf = 0.01, weight_decay = 0.001, save_period = 10, dropout = 0.15, cls = 0.5, box = 2.5,
            degrees = 5.0, translate = 0.2, scale = 0.4, mosaic = 0.25, mixup = 0.25, fliplr = 0.5,
            resume = False
            )

## Evaluation

In [ ]:
# validation set

val_output_dir = output_dir + '/val_results'
conf = 0.239 # custom confidence threshold for validation

model = YOLO(output_dir + '/weights/best.pt')
results = model.val(data = 'ValidationData.yaml', imgsz = 2560, device = 'cuda', batch = 4, project = output_dir, name = str(conf), conf = conf)

In [ ]:
# test set

val_output_dir = output_dir + '/val_results'
conf = 0.239 # custom confidence threshold for validation

model = YOLO(output_dir + '/weights/best.pt')
results = model.val(data = 'TestData.yaml', imgsz = 2560, device = 'cuda', batch = 4, project = output_dir, name = str(conf), conf = conf) # custom iou threshold can be added if needed with iou = 0.xx

## Inference and visualization

In [ ]:
model = YOLO(output_dir + '/weights/best.pt')
conf = 0.239 # 0.5 for high confidence threshold, as in paper

imagePath = 'YourKUBImage.png'  # replace with your image path

# Apply CLAHE preprocessing before inference - to match the training data preprocessing

clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8,8))
img = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)
img = clahe.apply(img)

results = model(img, imgsz = 2560, conf = conf, device = 'cuda')
for result in results:
  print(result.boxes)
  result.show()